# 演習8

In [1]:
ENV["LINES"] = 10
ENV["COLUMNS"] = 1000

1000

In [16]:
using DataFrames, DataFramesMeta, CSV, Dates, StatsBase, CategoricalArrays, Distributions, GLM

In [38]:
# 演習2
return_data_df = CSV.read("../../data/ch08_return_data.csv", DataFrame, missingstring=["NA"])
market_return_data_df = CSV.read("../../data/ch08_market_return_data.csv", DataFrame, missingstring=["NA"])
event_data_df = CSV.read("../../data/ch08_event_data.csv", DataFrame, missingstring=["NA"])

## 予想サプライズの計算とグルーピング
event_data_df = @chain event_data_df begin
    @transform(:event_ID = 1:size(event_data_df)[1])
    @rtransform begin
        :surp = (:earnings_forecast - :realized_earnings) / :lagged_ME
        :year = Dates.Year(:event_date).value
    end
    groupby(:year)
    @transform(:surp_rank = cut(:surp, 5, labels=1:5))
end

## 日付インデックスを作成
N_DAYS = market_return_data_df.date |> unique |> length
N_FIRMS = return_data_df.firm_ID |> unique |> length
N_EVENTS = event_data_df.event_date |> unique |> length

date_ID_df = DataFrame(
    :date => market_return_data_df.date |> unique |> sort,
    :date_ID => 1:N_DAYS,
)

market_return_data_df = innerjoin(market_return_data_df, date_ID_df, on = :date)

return_data_df = @chain return_data_df begin
    innerjoin(date_ID_df, on = [:date])
    @select($(Not(:date)))
    innerjoin(market_return_data_df, on = [:date_ID])
end

event_data_df = @chain event_data_df begin
    innerjoin(date_ID_df, on = [:event_date => :date])
    @select(:firm_ID, :surp, :surp_rank, :date_ID, :event_date, :year, :event_ID)
end

full_sample_data_df = DataFrame(
    :event_ID => repeat(1:N_EVENTS, (100 + 30 + 1 + 30)) |> sort,
    :relative_days => repeat(-(100 + 30):30, N_EVENTS)
)

event_data_df = @chain event_data_df begin
    innerjoin(full_sample_data_df, on = :event_ID)
    rename(:date_ID => :base_date_ID)
    @rtransform begin
        :date_ID = :base_date_ID + :relative_days + 1
    end
    innerjoin(return_data_df, on = [:firm_ID, :date_ID])
end

## ベータ推計
function estimate_beta(market_returns, returns)
    df = DataFrame(:R => returns, :R_M => market_returns)
    mdl = lm(@formula(R ~ R_M), df)
    return coef(mdl)
end

beta_df = @chain event_data_df begin
    @rsubset(:relative_days < -30)
    groupby(:firm_ID)
    @combine begin
        $AsTable = (:alpha, :beta = estimate_beta(:R_M, :R))
    end
end

## CARを計算
@chain event_data_df begin
    @rsubset(:relative_days >= -30)
    innerjoin(beta_df, on = :firm_ID)
    @rtransform(:AR = :R - :beta * :R_M)
end

ArgumentError: ArgumentError: column name :alpha not found in the data frame

,firm_ID,beta
,Int64,Float64
1,1,1.14012
2,2,0.303635
3,3,1.22318
4,4,0.886377
5,5,0.840162
6,6,1.06973
7,7,0.751604
8,8,1.33021
9,9,1.28661


estimate_beta (generic function with 2 methods)

In [27]:
mdl = estimate_beta(event_data_df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

R ~ 1 + R_M

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error       t  Pr(>|t|)     Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)  -6.11084e-5  2.44565e-5   -2.50    0.0125  -0.000109043  -1.31742e-5
R_M           0.992691    0.00231592  428.64    <1e-99   0.988152      0.997231
─────────────────────────────────────────────────────────────────────────────────

In [32]:
coef(mdl)[2]

0.9926913378133373